In [ ]:
import json
import re
from clovaspeechclient import ClovaSpeechClient
from collections import Counter
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
from glob import glob
import cv2
from tqdm import tqdm
import dlib
import numpy as np
import math

In [ ]:
file_id = 'KETI_MULTIMODAL_0000001228'

# @@@학습영상 추출@@@

In [ ]:
def lets_study(file_id):
    
    %cd '/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study'
    %cd {file_id}

    ### STT 정확도 높은 장면 ###
    
    # 대본 추출
    json_object = json.load(open(file_id + '_interpolation.json'))
    json_dialogue = json_object['dialogue_infos']

    st_time = {}
    for idx, dialog in enumerate(json_dialogue):
        st_time[idx] = dialog['start_time']
    sort_time = sorted(st_time.items(), key = lambda item: item[1])
    sort_time = [i[0] for i in sort_time]
    json_dialogue = [json_dialogue[i] for i in sort_time]       # 시간 순서대로 대사 정렬

    script_lst = []
    script = ''
    for d in json_dialogue:
        script = d['utterance']
        script = re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣]', '', script)
        script_lst.append(script)  
    script_len = len(script_lst)
    print(f'==========해당 영상의 총 대사 수는 {script_len}개 입니다.==========\n')
    print(f'대본 추출 결과 ==> {script_lst}')
      
    # STT 추출
    res = ClovaSpeechClient().req_upload(file=file_id + '.mp4', completion='sync')
    json_object = res.json()
    stt_lst = []
    for seg in json_object['segments']:
        stt = seg['text']
        stt = re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣]', '', stt)
        stt_lst.append(stt)
    print(f'STT 추출 결과 ==> {stt_lst}')
    
    # 대본, STT 비교
    i = -1
    valid_video_lst = []
    for sc, stt in zip(script_lst, stt_lst):
        i += 1
        if len(sc) >= 10:                                       # 글자수 10개 이상인 대본만 추출
            lst = [Counter(sc), Counter(stt)]
            df = pd.DataFrame(lst)
            acc = df.isna().sum(axis=1)[0] / len(Counter(sc))
            if acc < 5:                                         # 정확도 95% 이상인 대본만 추출
                valid_video_lst.append(i)
    print(f'{script_len}개의 영상 중 글자수가 10개 이상이며, STT 정확도가 높은 영상은 ==> {valid_video_lst}입니다.')

    # 폴더 만들고 영상 저장               
    lets_study = []
    lets_study_lip_point_lst = []
    for i in tqdm(valid_video_lst):
        s = json_dialogue[i]['start_time']
        e = json_dialogue[i]['end_time']
        s = int(s[:2])*3600 + int(s[3:5])*60 + float(s[6:])
        e = int(e[:2])*3600 + int(e[3:5])*60 + float(e[6:])
        try:
            os.mkdir(str(i))
            path = str(i) + '/'
            ffmpeg_extract_subclip(file_id + '.mp4', s, e, path + str(i) + '.mp4')
        except:
            print(f'{str(i)} 폴더가 이미 존재합니다.')
            pass
        

        ### 입술 좌표 추출되는 장면 ###

        # 영상을 프레임으로 저장
        path = str(i) + '/'
        video_cap = cv2.VideoCapture(path + str(i) + '.mp4')
        cnt = 0
        try:
            while video_cap.isOpened():
                ret, img = video_cap.read()
                cv2.imwrite(path + '%d.jpg' % cnt, img)
                cnt += 1
        except:
            video_cap.release()
            pass

        # 입술 좌표 추출 가능 여부 확인 및 추출
        print(f'{i}번 영상 확인을 시작합니다.')
        frame_lst = sorted(glob(str(i) + '/*jpg'), key=os.path.getctime)
        valid_frame_lst = []
        index = list(range(48, 68))
        lip_point_lst = []
        lip_cnt = 0
        for fidx, frame in enumerate(frame_lst):
            img = cv2.imread(frame)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            face_detector = dlib.get_frontal_face_detector()
            faces = face_detector(img_gray)
            if len(faces) == 0:
                lip_point = []
                lip_point_lst.append(lip_point)
                lip_cnt += 1
                if lip_cnt > len(frame_lst) * 0.20:
                    print(f'{i}번 영상의 {frame}는 입술 좌표 추출이 불가합니다. {i}번 영상 확인을 종료합니다.')
                    break
            else:
                face_size = 0
                num = 0
                for n, face in enumerate(faces):
                    if face.area() > face_size:
                        face_size = face.area()
                        num = n
                landmark_model = dlib.shape_predictor('/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study/shape_predictor_68_face_landmarks.dat')
                lm = landmark_model(img, faces[num])
                lm_point = [[p.x, p.y] for p in lm.parts()]
                lm_point = np.array(lm_point)
                lip_point = [tuple(lm_point[index][i]) for i in range(len(index))]
                valid_frame_lst.append(fidx)
                lip_point_lst.append(lip_point)
                if fidx + 1 == len(frame_lst):
                    lets_study.append(i)
                    lets_study_lip_point_lst.append(lip_point_lst)
                    print(f"{i}번 영상은 Let's study 학습 자료로 활용이 가능합니다.")
                    print(f'{i}번 영상확인 결과==========> {valid_frame_lst}')

    print(f'모든 영상에 대한 확인이 끝났습니다.')
    print("==========다음 영상을 Let's study 학습 자료로 활용하세요==========")
    print(lets_study)

    return lets_study, lets_study_lip_point_lst

In [ ]:
lets_study, lets_study_lip_point_lst = lets_study(file_id)

/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study
/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study/KETI_MULTIMODAL_0000001228
==========해당 영상의 총 대사 수는 8개 입니다.==========

대본 추출 결과 ==> ['어때어숯의불향이깊게스며들어서담백하면서도겉은바삭하고속은촉촉한환상의궁합이', '하나진짜인정하고싶지않은데이것도맛있다', '거봐어내가왜식샤님이라고불리겠어', '식샤님은개뿔이건삼겹살님의은혜거든역시삼겹살님은언제나옳아', '그렇게말하면여기있는이소세지님어채소님조개님이거얼마나섭섭해하시겠어안그래', '섭섭하셨어요흠제가곧먹어드릴께요', '먹자씨', '어휴야나보기만해도너무행복해']
STT 추출 결과 ==> ['어때어숯의불량이깊게스며들어서담백하면서도겉은바삭하고속은촉촉한환상의궁합이', '아나진짜인정하고싶지않은데이것도맛있다', '거봐어내가왜식자님이라고불리겠어', '식시한님은갯불이건삼겹살님의은혜거든역시삼겹살님은언제나오라', '응그렇게말하면여기있는이소세지님어채소님조개님이거얼마나섭섭해하시겠어안그래', '섭섭하셨어요제가곧먹어드릴게요', '짜식', '아우야나보기만해도너무행복해']
8개의 영상 중 글자수가 10개 이상이며, STT 정확도가 높은 영상은 ==> [0, 1, 2, 3, 4, 5, 7]입니다.


  0%|                                                     | 0/7 [00:00<?, ?it/s]

0 폴더가 이미 존재합니다.
0번 영상 확인을 시작합니다.


 14%|██████▍                                      | 1/7 [01:17<07:44, 77.34s/it]

0번 영상의 0/136.jpg는 입술 좌표 추출이 불가합니다. 0번 영상 확인을 종료합니다.
1 폴더가 이미 존재합니다.
1번 영상 확인을 시작합니다.


...

4번 영상의 4/83.jpg는 입술 좌표 추출이 불가합니다. 4번 영상 확인을 종료합니다.
5 폴더가 이미 존재합니다.
5번 영상 확인을 시작합니다.


 86%|██████████████████████████████████████▌      | 6/7 [06:36<01:07, 67.71s/it]

5번 영상은 Let's study 학습 자료로 활용이 가능합니다.
5번 영상확인 결과==========> [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]
7 폴더가 이미 존재합니다.
7번 영상 확인을 시작합니다.


100%|█████████████████████████████████████████████| 7/7 [06:44<00:00, 57.80s/it]

7번 영상의 7/22.jpg는 입술 좌표 추출이 불가합니다. 7번 영상 확인을 종료합니다.
모든 영상에 대한 확인이 끝났습니다.
==========다음 영상을 Let's study 학습 자료로 활용하세요==========
[3, 5]


In [ ]:
lets_study_lip_point_lst

[[[(1141, 501),
   (1163, 490),
   (1184, 485),
   (1197, 492),
   (1210, 491),
   (1224, 503),
   (1237, 518),
   (1219, 532),
   (1202, 537),
   (1188, 536),
   (1173, 532),
   (1155, 522),
   (1149, 503),
   (1181, 504),
   (1194, 507),
   (1207, 509),
   (1228, 517),
   (1205, 513),
   (1192, 512),
   (1179, 509)],
  [(1140, 501),
   (1163, 491),
   (1184, 486),
   (1197, 492),
   (1210, 491),
   (1225, 503),
   (1238, 518),
   (1219, 532),
   (1203, 537),
   (1188, 536),
   (1173, 532),
   (1156, 522),
   (1149, 503),
   (1181, 505),
   (1194, 508),
   (1207, 510),
   (1228, 517),
   (1206, 513),
   (1192, 512),
   (1179, 508)],
  [(1148, 504),
   (1171, 495),
   (1192, 490),
   (1205, 496),
   (1217, 495),
   (1231, 506),
   (1243, 521),
   (1225, 536),
   (1210, 541),
   (1196, 540),
   (1182, 537),
   (1164, 526),
   (1157, 507),
   (1190, 508),
   (1202, 511),
   (1215, 512),
   (1233, 520),
   (1212, 516),
   (1200, 516),
   (1187, 512)],
  [(1157, 508),
   (1180, 499),
   (1

# @@@입술 분석@@@

In [ ]:
video_idx = input(f'{file_id}의 몇 번째 영상을 학습하시겠습니까? ==> ')

KETI_MULTIMODAL_0000001228의 몇 번째 영상을 학습하시겠습니까? ==> 0


In [ ]:
def lip_pronunciation_analysis(video_idx, lets_study_lip_point_lst):
    
    %cd '/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study'
    
    ### 원본 영상의 입술 좌표 ###
    lip_point_original = lets_study_lip_point_lst[int(video_idx)]
    
    
    ### 학습자 영상의 입술 좌표 ###
    
    # 입술 좌표 추출
    frame_lst = sorted(glob('test/*jpg'), key=os.path.getctime)
    index = list(range(48, 68))
    lip_point_test = []
    
    for frame in tqdm(frame_lst):
        img = cv2.imread(frame)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        face_detector = dlib.get_frontal_face_detector()
        faces = face_detector(img_gray)
        if len(faces) == 0:
            lip_point = []
        else:
            face_size = 0
            num = 0
            for n, face in enumerate(faces):
                if face.area() > face_size:
                    face_size = face.area()
                    num = n
            landmark_model = dlib.shape_predictor('/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study/shape_predictor_68_face_landmarks.dat')
            lm = landmark_model(img, faces[num])
            lm_point = [[p.x, p.y] for p in lm.parts()]
            lm_point = np.array(lm_point)
            lip_point = [tuple(lm_point[index][i]) for i in range(len(index))]

            # Translation
            lip_point_translation = [(lip_point[i][0]-lip_point[9][0], lip_point[i][1]-lip_point[9][1]) for i in range(20)]

            # Aligning
            rad = math.atan2(lip_point_translation[3][0], lip_point_translation[3][1])
            lip_point_aligning = []
            for i in range(20):
                lip_point_aligning.append((math.cos(rad)*lip_point_translation[i][0] - math.sin(rad)*lip_point_translation[i][1],
                                        math.sin(rad)*lip_point_translation[i][0] + math.cos(rad)*lip_point_translation[i][1]))
                
            # Scaling
            lip_point_scaling = lip_point_aligning / np.linalg.norm(lip_point_aligning)

            lip_point_test.append(lip_point_scaling)


    ### 프레임 단위로 입술 좌표 비교 ###
    print(f'원본 영상 프레임: {len(lip_point_original)}개, 학습자 영상 프레임: {len(lip_point_test)}개')
    if len(lip_point_original) == len(lip_point_test):
        print('원본 영상과 학습자 영상의 프레임 수가 일치합니다.')
        lip_point_original = np.array(lip_point_original)
        lip_point_test = np.array(lip_point_test)
        
        sum_lst = []
        for o, t in zip(lip_point_original, lip_point_test):
            try:
                sum_lst.append(np.square(o - t))
                score = math.sqrt(np.sum(sum_lst) / len(sum_lst))    ### 점수 산출 방법 다시 검토 ###
            except:
                pass
        print(f'입술 분석 결과: {score}')

    else:
        print('원본 영상과 학습자 영상의 프레임 수가 일치하지 않습니다.')
        print('분석을 종료합니다.')

In [ ]:
lip_pronunciation_analysis(video_idx, lets_study_lip_point_lst)

/Users/sjk/Desktop/멀캠/6. 왕누나있조/Lets_Study


100%|█████████████████████████████████████████| 201/201 [02:05<00:00,  1.60it/s]

원본 영상 프레임: 201개, 학습자 영상 프레임: 201개
원본 영상과 학습자 영상의 프레임 수가 일치합니다.
입술 분석 결과: 5636170269.297071
